In [ ]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt

#Classification with KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
#load data frames and give them a name
def loadDataFrames(name, pathAcc, pathGyr, pathOri):
    df_Acc= pd.read_csv(pathAcc)
    df_Gyr= pd.read_csv(pathGyr)
    df_Ori= pd.read_csv(pathOri)
    print("Data Frames for " + name + " loaded.")
    return df_Acc, df_Gyr, df_Ori	    

#load mixed data
df_mixed_Acc, df_mixed_Gyr, df_mixed_Ori =loadDataFrames('mixed','data/MixedData/Accelerometer.csv', 'data/MixedData/Gyroscope.csv', 'data/MixedData/Orientation.csv')

#load data frames for walking
df_walk_Acc, df_walk_Gyr, df_walk_Ori =loadDataFrames('walking','data/NormalWalk/Accelerometer.csv', 'data/NormalWalk/Gyroscope.csv', 'data/NormalWalk/Orientation.csv')

#load data frames for push ups
df_push_Acc, df_push_Gyr, df_push_Ori =loadDataFrames('push ups','data/PushUps/Accelerometer.csv', 'data/PushUps/Gyroscope.csv', 'data/PushUps/Orientation.csv')

#load data frames for jumping jacks
df_JJ_Acc, df_JJ_Gyr, df_JJ_Ori =loadDataFrames('jumping jacks','data/JJ_rightHand/Accelerometer.csv', 'data/JJ_rightHand/Gyroscope.csv', 'data/JJ_rightHand/Orientation.csv')




In [ ]:
df_push_Acc.head(10)
#ggf. spalten umbennen zb. z zu z_Acc etc. 
#ggf. Spalten dropen die ich nicht brauche wie yawn, pitch, roll oder so, mal schauen wie die Daten aussehen

In [ ]:
df_push_Gyr.plot(x='time', y='z', kind='line')
df_push_Gyr.plot(x='time', y='y', kind='line')
df_push_Gyr.plot(x='time', y='x', kind='line')

In [ ]:
#Plotting Gyroscope Data
#Function to plot Gyr Data
def plotDataGyr(data, title):
    plt.plot(data['time'], data['z'], color='red', marker='.')
    plt.plot(data['time'], data['y'], color='blue', marker='.')
    plt.plot(data['time'], data['x'], color='green', marker='.')
    plt.xlabel('time')
    plt.ylabel('x,y,z')
    plt.legend(['z', 'y', 'x'])
    plt.title(title)
    plt.show()

In [ ]:
plotDataGyr(df_mixed_Acc, 'Mixed Data')

In [ ]:
#Plot Walk
plotDataGyr(df_walk_Gyr, 'Gyroscope Walk')

#Plot PushUps
plotDataGyr(df_push_Gyr, 'Gyroscope PushUps')

#Plot JumpingJacks
plotDataGyr(df_JJ_Gyr, 'Gyroscope JumpingJacks')

In [ ]:
#Plotting Accelerometer Data
#Function to Plot Acc Data
def plotDataAcc(data, title):
    plt.plot(data['time'], data['z'], color='red', marker='.')
    plt.plot(data['time'], data['y'], color='blue', marker='.')
    plt.plot(data['time'], data['x'], color='green', marker='.')
    plt.xlabel('time')
    plt.ylabel('x,y,z')
    plt.legend(['z', 'y', 'x'])
    plt.title(title)
    plt.show()

#Plot Walk
plotDataAcc(df_walk_Acc, 'Accelerometer Walk')

#Plot PushUps
plotDataAcc(df_push_Acc, 'Accelerometer PushUps')

#Plot JumpingJacks
plotDataAcc(df_JJ_Acc, 'Accelerometer JumpingJacks')

In [ ]:
#Plotting Orientation Data
#Function to Plot Ori Data
def plotDataOri(data, title):
    plt.plot(data['time'], data['qz'], color='red', marker='.')
    plt.plot(data['time'], data['qy'], color='blue', marker='.')
    plt.plot(data['time'], data['qx'], color='green', marker='.')
    plt.plot(data['time'], data['qw'], color='yellow', marker='.')
    #plt.plot(data['time'], data['roll'], color='black', marker='.')
    plt.xlabel('time')
    plt.ylabel('qz,qy,qx,qw,roll')
    plt.legend(['qz', 'qy', 'qx', 'qw', 'roll'])
    plt.title(title)
    plt.show()

#Plot Walk
plotDataOri(df_walk_Ori, 'Orientation Walk')

#Plot PushUps
plotDataOri(df_push_Ori, 'Orientation PushUps')

#Plot JumpingJacks
plotDataOri(df_JJ_Ori, 'Orientation JumpingJacks')
    

In [ ]:
#Concat DataFrames Walk
df_walk_Gyr.drop(['time','seconds_elapsed'], axis=1, inplace=True)
df_walk_Ori.drop(['time','seconds_elapsed'], axis=1, inplace=True)
df_walk = pd.concat([df_walk_Acc, df_walk_Gyr, df_walk_Ori], axis=1)

In [ ]:
#Concat Mixed DataFrames
df_mixed_Gyr.drop(['time','seconds_elapsed'], axis=1, inplace=True)
df_mixed_Ori.drop(['time','seconds_elapsed'], axis=1, inplace=True)
df_mixed = pd.concat([df_mixed_Acc, df_mixed_Gyr, df_mixed_Ori], axis=1)
df_mixed.head(10)

In [ ]:
df_walk.describe()


In [ ]:
#Concat DataFrames PushUps
df_push_Gyr.drop(['time','seconds_elapsed'], axis=1, inplace=True)
df_push_Ori.drop(['time','seconds_elapsed'], axis=1, inplace=True)
df_push = pd.concat([df_push_Acc, df_push_Gyr, df_push_Ori], axis=1)
df_push.head(10)

In [ ]:
#Concat DataFrames JumpingJacks
df_JJ_Gyr.drop(['time','seconds_elapsed'], axis=1, inplace=True)
df_JJ_Ori.drop(['time','seconds_elapsed'], axis=1, inplace=True)
df_JJ = pd.concat([df_JJ_Acc, df_JJ_Gyr, df_JJ_Ori], axis=1)
df_JJ.head(-20)

In [ ]:
#Classification with KNN
import pickle as pkl
#Function to label the data
def getData(data, label):
    data['label'] = label
    return data

#Label Data Walk
df_walk = getData(df_walk, 'walk')
#df_walk.head(10)

#Label Data PushUps
df_push = getData(df_push, 'pushups')
#df_push.head(10)

#Label Data JumpingJacks
df_JJ = getData(df_JJ, 'jumpingjacks')
#df_JJ.head(10)


#Concat DataFrames
df = pd.concat([df_walk, df_push, df_JJ], axis=0)

#Fill NaN Values with propagation
df.fillna(method='ffill', inplace=True)
df_mixed.fillna(method='ffill', inplace=True)

#Split Data
X = df.drop(['label'], axis=1)
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

#gemischten Datensatz vlt mal testen, weiß noch nicht genau wie ich den einbauen soll


#pickle wird benutzt um das trainierte Modell zu speichern und dann in streamlit zu laden
#pkl.dump(knn, open('knn.pkl', 'wb'))

In [1]:
#KNN
knn = KNeighborsClassifier(n_neighbors=3, weights='distance')
knn.fit(X_train, y_train)
y_pred = knn.predict(df_mixed)

#size passt noch nicht, da der ytest mehr data hat als der ypred

NameError: name 'KNeighborsClassifier' is not defined

In [ ]:
#Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

#Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)


In [ ]:
#Label Data Mixed
df_mixed = getData(df_mixed, 'mixed')
#df_mixed.head(10)

#Split Data Mixed
X_mixed = df_mixed.drop(['label'], axis=1)
y_mixed = df_mixed['label']
X_train_mixed, X_test_mixed, y_train_mixed, y_test_mixed = train_test_split(X_mixed, y_mixed, test_size=0.3)

#KNN
knn = KNeighborsClassifier(n_neighbors=3, weights='distance')
knn.fit(X_train, y_train)
y_pred_mixed = knn.predict(X_test)